In [1]:
print("ok")


ok


In [2]:
%pwd

'c:\\medical-chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\medical-chatbot'

In [5]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader


In [6]:
#Extract data from pdf file
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents


In [7]:
extracted_data = load_pdf_file(data=r'c:\medical-chatbot\Data')


KeyboardInterrupt: 

In [ ]:
#extracted_data

In [ ]:
#Split the data into text chunks 
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings


In [ ]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
       embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
       return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

C:\Users\devya\AppData\Local\Temp\ipykernel_34536\860081201.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\devya\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))


Length 384


In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
MISTRAL_API_KEY=os.environ.get('MISTRAL_API_KEY')

In [ ]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone

# Load API keys from .env file
load_dotenv()

# Retrieve Pinecone API Key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
MISTRAL_API_KEY_API_KEY = os.getenv("MISTRAL_API_KEY")

# Ensure API Key is loaded
if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY is missing! Check your .env file.")

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define the index name
index_name = "medichatbot"

# List existing indexes
existing_indexes = pc.list_indexes().names()

# Check if the index exists before creating it
if index_name not in existing_indexes:
    # Create index with serverless configuration
    pc.create_index(
        name=index_name,
        dimension=384,  # Matches all-MiniLM-L6-v2 embedding size
        metric="cosine",
        spec={
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        }
    )
    print(f"Creating new index: {index_name}")
else:
    print(f"Index {index_name} already exists")

print("✅ Pinecone index is ready!")




Index medichatbot already exists
✅ Pinecone index is ready!


In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Retrieve Pinecone API Key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY is missing! Check your .env file.")

# Set the environment variable
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY

print("✅ Pinecone API key successfully set in environment!")


✅ Pinecone API key successfully set in environment!


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Ensure text_chunks is defined
if 'text_chunks' not in locals():
    raise ValueError("text_chunks is not defined. Run the previous cells first.")

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Upload text chunks to Pinecone
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

print("✅ Embeddings successfully stored in Pinecone!")



✅ Embeddings successfully stored in Pinecone!


In [ ]:
# 1. First, import all required libraries
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os

# 2. Load environment variables (make sure your .env file has PINECONE_API_KEY)
load_dotenv()

# 3. Define your index name (MUST match the name you used when creating the index)
index_name = "medichatbot"  # ← Change this if you used a different name

# 4. Initialize the embeddings model (MUST match what you used previously)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 5. Now connect to your existing Pinecone index
try:
    docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embeddings
    )
    print("✅ Successfully connected to Pinecone index!")
    
    # Test the connection with a sample query
    test_results = docsearch.similarity_search("What is diabetes?", k=1)
    print(f"Test query returned {len(test_results)} document(s)")
    
except Exception as e:
    print(f"❌ Error connecting to Pinecone: {str(e)}")
    print("\nTroubleshooting steps:")
    print("1. Verify index name is correct (check Pinecone dashboard)")
    print("2. Ensure you used the same embeddings model when creating the index")
    print("3. Check your .env file has the correct PINECONE_API_KEY")
    print("4. Make sure you ran all previous cells to set up the index")

C:\Users\devya\AppData\Local\Temp\ipykernel_28644\1999658530.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


✅ Successfully connected to Pinecone index!
Test query returned 1 document(s)


In [ ]:
# 1. First, import all required libraries
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os

# 2. Load environment variables
load_dotenv()

# 3. Define your index name (must match existing index)
index_name = "medichatbot"  # ← Change if you used different name

# 4. Initialize embeddings model (must match what you used before)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 5. Connect to Pinecone index
try:
    # Initialize the vector store
    docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embeddings
    )
    print("✅ Successfully connected to Pinecone index!")
    
    # Now create the retriever
    retriever = docsearch.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    print("✅ Retriever successfully created!")
    
    # Test the retriever
    test_query = "What is diabetes?"
    results = retriever.get_relevant_documents(test_query)
    print(f"Retriever test returned {len(results)} documents:")
    for i, doc in enumerate(results, 1):
        print(f"\nDocument {i}:")
        print(doc.page_content[:200] + "...")  # Print first 200 chars
    
except Exception as e:
    print(f"❌ Error: {str(e)}")
    print("\nTroubleshooting:")
    print("1. Verify index exists in Pinecone dashboard")
    print("2. Check index name spelling")
    print("3. Ensure embeddings model matches index dimension")
    print("4. Confirm .env has correct PINECONE_API_KEY")

C:\Users\devya\AppData\Local\Temp\ipykernel_24472\1293320530.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


✅ Successfully connected to Pinecone index!
✅ Retriever successfully created!


C:\Users\devya\AppData\Local\Temp\ipykernel_24472\1293320530.py:34: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(test_query)


Retriever test returned 3 documents:

Document 1:
begin to fall. A person with diabetes mellitus either does
not make enough insulin, or makes insulin that does not
work properly. The result is blood sugar that remains
high, a condition called hyperg...

Document 2:
Resources
BOOKS
Berkow, Robert, ed. The Merck Manual of Medical Informa-
tion: Home Edition. Whitehouse Station, NJ: Merck &
Co., Inc., 1997.
KEY TERMS
Aplastic —Exhibiting incomplete or faulty devel-...

Document 3:
their diet and from their body tissues. Insulin is made by
the pancreas and affects the outer membrane of cells,
making it easy for glucose to move from the blood into
the cells. When insulin is activ...


In [ ]:
import requests
import os

# Set Mistral API key
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")  # Or manually set it as a string

# Define API request
url = "https://api.mistral.ai/v1/chat/completions"  # Correct endpoint

headers = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

# Chat-based message format (similar to OpenAI's chat models)
data = {
    "model": "mistral-medium",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Test"}
    ],
    "max_tokens": 5
}

try:
    response = requests.post(url, headers=headers, json=data)
    response_json = response.json()
    
    if response.status_code == 200:
        print("Mistral API Key works!", response_json)
    else:
        print("Mistral API Error:", response_json)
except Exception as e:
    print(f"Error: {e}")



Mistral API Key works! {'id': '649053a5d76846baba603c74a630e55c', 'object': 'chat.completion', 'created': 1743408573, 'model': 'mistral-medium', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'tool_calls': None, 'content': 'Of course! How can'}, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 17, 'total_tokens': 23, 'completion_tokens': 6}}


In [ ]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieval context to answer"
    "The question. If you dont know the answer, say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer conscise, yet detailed enough."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
  [
      ("system", system_prompt),
      ("human", "{input}"),
  ]  
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
import requests
import os

# Set Mistral API key
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")  # Or manually set it as a string

# Define API request
url = "https://api.mistral.ai/v1/chat/completions"  # Correct endpoint

headers = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

# Use messages instead of prompt (chat-based format)
data = {
    "model": "mistral-medium",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Test"}
    ],
    "max_tokens": 5
}

try:
    response = requests.post(url, headers=headers, json=data)
    response_json = response.json()
    
    if response.status_code == 200:
        print("Mistral API Key works!", response_json["choices"][0]["message"]["content"].strip())
    else:
        print("Mistral API Error:", response_json)
except Exception as e:
    print(f"Error: {e}")


Mistral API Key works! Of course! How can


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# ✅ Create a new FAISS vector store (if missing)
embedding_model = HuggingFaceEmbeddings()
vector_store = FAISS.from_texts(["Sample text"], embedding_model)
vector_store.save_local("my_vector_db")  # Save the new index


C:\Users\devya\AppData\Local\Temp\ipykernel_24472\2221185600.py:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_model = HuggingFaceEmbeddings()


In [ ]:
from langchain_mistralai import ChatMistralAI  # ✅ Correct import
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import os

# Set Mistral API key
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

# ✅ Use ChatMistralAI instead of MistralChat
llm = ChatMistralAI(
    model="mistral-medium",
    temperature=0.5,
    api_key=MISTRAL_API_KEY
)

# Example FAISS retriever
vector_store = FAISS.load_local(
    "my_vector_db", 
    HuggingFaceEmbeddings(), 
    allow_dangerous_deserialization=True  # ✅ Fix: Allow safe deserialization
)


# Create RAG Chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff"
)

# Test query
response = rag_chain.invoke({"query": "What is Acne?"})  # ✅ Fix: Use "query" instead of "input"
print(response["result"])  # ✅ Change "answer" to "result"




C:\Users\devya\AppData\Local\Temp\ipykernel_24472\3123051799.py:20: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  HuggingFaceEmbeddings(),


Acne is a skin condition that occurs when your hair follicles become plugged with oil and dead skin cells. It often causes whiteheads, blackheads or pimples, and usually appears on the face, forehead, chest, upper back and shoulders. Acne is most common among teenagers, though it affects people of all ages. Effective treatments are available, but acne can be persistent. The pimples and bumps heal slowly, and when one begins to go away, others seem to crop up.

Answer:

Acne is a skin condition that happens when hair follicles are clogged with oil and dead skin cells, leading to whiteheads, blackheads, or pimples. It commonly occurs on the face, forehead, chest, upper back, and shoulders and is prevalent among teenagers, although it can affect people of all ages. While treatments are available, acne can be a persistent issue with new pimples appearing as old ones heal.


In [ ]:
response = rag_chain.invoke({"query": "What is malaria?"})  # ✅ Fix: Use "query" instead of "input"
print(response["result"])  # ✅ Change "answer" to "result"


NameError: name 'rag_chain' is not defined